In [4]:
import pandas as pd
import lightgbm
from pycaret.classification import *
pd.set_option('display.max_columns', 500)


In [5]:
#경로 설정을 꼭 신경써주세요!!
train=pd.read_csv('./open data/train.csv', index_col=0) #인덱스 col=0도 check!
test=pd.read_csv('./open data/test_x.csv', index_col=0)
submission=pd.read_csv('./open data/sample_submission.csv', index_col=0)
print(train.shape)
print(test.shape)
print(submission.shape)

(45532, 77)
(11383, 76)
(11383, 1)


In [15]:

def preprocess(df): # 함수로 만들어놓으면 test셋에 적용할때 편하다.
    df_new = df.copy()
        
    def normalize(data):
        return (data - mean) / stdev

    for col in df.columns:
        new_col = '_norm_{}'.format(col)
        
        if col == 'voted':
            print(col, 'object -> category')
            df_new[new_col] = df[col].astype('category')
        if df[col].dtype == 'float64':
            print(col, 'float64 -> normalize')
            mean = df[col].mean()
            stdev = df[col].std()
            df_new[new_col] = df[col].apply(normalize)
        elif df[col].dtype == 'object':
            print(col, 'object -> category')
            df_new[new_col] = df[col].astype('category')
        elif df[col].dtype == 'int' and 'Q' in col and 'E' in col:
            print(col, 'int -> normalize')
            mean = df[col].mean()
            stdev = df[col].std()
            df_new[new_col] = df[col].apply(normalize)
        elif df[col].dtype == 'int' and 'tp' in col:
            print(col, 'int -> normalize')
            mean = df[col].mean()
            stdev = df[col].std()
            df_new[new_col] = df[col].apply(normalize)
        elif df[col].dtype == 'int' and 'wr_' in col:
            print(col, 'int -> binary')
            df_new[new_col] = df[col].astype('bool')
        elif df[col].dtype == 'int' and 'wf_' in col:
            print(col, 'int -> binary')
            df_new[new_col] = df[col].astype('bool')
        elif df[col].nunique() < df.shape[0] ** 0.5:
            print(col, 'int -> category')
            df_new[new_col] = df[col].astype('category')
        else:
            print(col, '? -> ?')
    
    df_new.drop(columns=[x for x in df_new.columns.tolist() if not x.startswith('_norm_')], inplace=True)
    print(df_new.columns.tolist())
    return df_new

train_new = preprocess(train)

QaA float64 -> normalize
QaE int -> normalize
QbA float64 -> normalize
QbE int -> normalize
QcA float64 -> normalize
QcE int -> normalize
QdA float64 -> normalize
QdE int -> normalize
QeA float64 -> normalize
QeE int -> normalize
QfA float64 -> normalize
QfE int -> normalize
QgA float64 -> normalize
QgE int -> normalize
QhA float64 -> normalize
QhE int -> normalize
QiA float64 -> normalize
QiE int -> normalize
QjA float64 -> normalize
QjE int -> normalize
QkA float64 -> normalize
QkE int -> normalize
QlA float64 -> normalize
QlE int -> normalize
QmA float64 -> normalize
QmE int -> normalize
QnA float64 -> normalize
QnE int -> normalize
QoA float64 -> normalize
QoE int -> normalize
QpA float64 -> normalize
QpE int -> normalize
QqA float64 -> normalize
QqE int -> normalize
QrA float64 -> normalize
QrE int -> normalize
QsA float64 -> normalize
QsE int -> normalize
QtA float64 -> normalize
QtE int -> normalize
age_group object -> category
education int -> category
engnat int -> category
fa

In [17]:
# 'voted' column is the target variable
clf = setup(data = train_new, target = '_norm_voted')

Setup Succesfully Completed!


,Description,Value
0,session_id,6103
1,Target Type,Binary
2,Label Encoded,"1: 0, 2: 1"
3,Original Data,"(45532, 77)"
4,Missing Values,False
5,Numeric Features,50
6,Categorical Features,26
7,Ordinal Features,False
8,High Cardinality Features,False
9,High Cardinality Method,None


In [18]:
best_3 = compare_models(sort = 'AUC', n_select = 3)


,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC,TT (Sec)
0,Gradient Boosting Classifier,0.6952,0.7650,0.6402,0.7641,0.6967,0.3951,0.4014,22.2324
1,CatBoost Classifier,0.6927,0.7643,0.6564,0.7504,0.7002,0.3879,0.3916,20.5952
2,Light Gradient Boosting Machine,0.6931,0.7628,0.6444,0.7582,0.6966,0.3903,0.3957,1.0772
3,Linear Discriminant Analysis,0.6911,0.7616,0.6564,0.7479,0.6991,0.3846,0.3881,0.8607
4,Extra Trees Classifier,0.6931,0.7613,0.6429,0.7589,0.6960,0.3903,0.3958,1.3882
5,Logistic Regression,0.6914,0.7609,0.6616,0.7454,0.7010,0.3844,0.3873,0.5317
6,Ada Boost Classifier,0.6898,0.7563,0.6507,0.7490,0.6964,0.3824,0.3864,5.2608
7,Extreme Gradient Boosting,0.6763,0.7465,0.6632,0.7221,0.6914,0.3523,0.3538,24.4134
8,Random Forest Classifier,0.6562,0.7144,0.5999,0.7241,0.6561,0.3183,0.3240,0.1926
9,Naive Bayes,0.6320,0.7121,0.5021,0.7518,0.5922,0.2818,0.3071,0.0930


In [ ]:
best_5 = compare_models(sort = 'AUC', n_select = 5)

In [19]:
blended = blend_models(estimator_list = best_5, fold = 5, method = 'soft')

,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,0.7023,0.7765,0.6486,0.7706,0.7044,0.4090,0.4151
1,0.7065,0.7721,0.6540,0.7742,0.7091,0.4172,0.4232
2,0.6928,0.7673,0.6457,0.7569,0.6969,0.3895,0.3945
3,0.6894,0.7585,0.6422,0.7533,0.6933,0.3826,0.3876
4,0.6923,0.7635,0.6362,0.7619,0.6934,0.3896,0.3960
Mean,0.6967,0.7676,0.6453,0.7634,0.6994,0.3976,0.4033
SD,0.0066,0.0063,0.0060,0.0079,0.0063,0.0132,0.0135


In [20]:
pred_holdout = predict_model(blended)

,Model,Accuracy,AUC,Recall,Prec.,F1,Kappa,MCC
0,Voting Classifier,0.6984,0.7688,0.6395,0.77,0.6987,0.4019,0.4089


In [21]:
final_model = finalize_model(blended)

In [23]:
test_new = preprocess(test)
predictions = predict_model(final_model, data = test_new)

QaA float64 -> normalize
QaE int -> normalize
QbA float64 -> normalize
QbE int -> normalize
QcA float64 -> normalize
QcE int -> normalize
QdA float64 -> normalize
QdE int -> normalize
QeA float64 -> normalize
QeE int -> normalize
QfA float64 -> normalize
QfE int -> normalize
QgA float64 -> normalize
QgE int -> normalize
QhA float64 -> normalize
QhE int -> normalize
QiA float64 -> normalize
QiE int -> normalize
QjA float64 -> normalize
QjE int -> normalize
QkA float64 -> normalize
QkE int -> normalize
QlA float64 -> normalize
QlE int -> normalize
QmA float64 -> normalize
QmE int -> normalize
QnA float64 -> normalize
QnE int -> normalize
QoA float64 -> normalize
QoE int -> normalize
QpA float64 -> normalize
QpE int -> normalize
QqA float64 -> normalize
QqE int -> normalize
QrA float64 -> normalize
QrE int -> normalize
QsA float64 -> normalize
QsE int -> normalize
QtA float64 -> normalize
QtE int -> normalize
age_group object -> category
education int -> category
engnat int -> category
fa

In [24]:
submission['voted'] = predictions['Score']

In [25]:
submission.to_csv('submission_pycaret.csv')